# Car Racing Baseline (无域适应)

Baseline 版本：只使用 PPO 训练，不使用域适应方法。

用于对比 DANN 方法的有效性。

## 1. 克隆仓库并导入模块

In [ ]:
# Git Clone 仓库并导入所有模块
!git clone https://github.com/bochendong/car-racing-revistied.git 2>/dev/null || echo "Repository already exists or clone failed"
%cd car-racing-revistied

# 导入 baseline 模块（使用新的模块化结构）
import sys
sys.path.append('src')

from src.utils import Env, eval, ExperimentLogger
from src.models import BaselineModel
from src.agents import BaselineAgent

print("✅ Baseline 模块导入成功！")


## 2. 环境设置

In [ ]:
# Colab 环境设置
%%capture
%pip install swig
!sudo apt update && sudo apt install python-opengl
!sudo apt update && sudo apt install xvfb
%pip install gym-notebook-wrapper stable-baselines[mpi] pyglet
%pip install pyvirtualdisplay -qq
!apt-get install -y xvfb python-opengl ffmpeg -qq
%pip install gym[box2d]
%pip install box2d-kengz

## 3. 导入必要的库

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import gym

## 4. 设备设置和输出目录

In [ ]:
# 设置设备并创建输出目录
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f"使用设备: {device}")

if not os.path.exists("./output_r"):
    os.mkdir("output_r")
    
# 清空输出目录
for epoch in range(3000):
    files = glob.glob("./output_r/*.png")
    for f in files:
        os.remove(f)

## 5. 初始化环境和模型

In [ ]:
# 创建环境
green_env = Env(color='g', seed=0)
env_c1 = Env(color='c1', seed=0)
env_c2 = Env(color='c2', seed=0)

source_env = green_env
target_env = [env_c1, env_c2]

In [ ]:
# 初始化实验记录器
from datetime import datetime
import json
experiment_name = f"baseline_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
logger = ExperimentLogger(experiment_name=experiment_name, base_dir="./experiments")
print(f"✅ 实验记录器初始化完成！实验目录: {logger.experiment_dir}")

# 记录实验配置
config = {
    "method": "Baseline (PPO only)",
    "model": "BaselineModel",
    "learning_rate": 1e-4,
    "buffer_capacity": 1024,
    "batch_size": 128,
    "ppo_epoch": 10,
    "device": str(device),
    "total_episodes": 3000,
    "domain_adaptation": False
}
logger.log_config(config)

# 初始化 baseline 模型和智能体（不使用域适应）
net = BaselineModel().double().to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-4)

# Baseline Agent 不需要 criterion（没有域分类器）
agent = BaselineAgent(net=net, optimizer=optimizer, 
                      buffer_capacity=1024, batch_size=128, device=device)

print("✅ Baseline 模型和智能体初始化完成！")

## 6. 训练（Baseline）

In [ ]:
training_records = []
running_score_records = []
running_score = 0

c1_training_records = []
c2_training_records = []

for i_ep in range(3000):
    score = 0
    state = source_env.reset()

    for t in range(1000):
        action, a_logp = agent.select_action(state)
        state_, reward, done, die = source_env.step(action * np.array([2., 1., 1.]) + np.array([-1., 0., 0.]))
        score += reward

        should_update = agent.store((state, action, a_logp, reward, state_))

        if should_update:
            agent.update(epoch=i_ep)  # Baseline 不需要 eta 参数

        state = state_

        if done or die:
            break

    # 记录分数和计算移动平均
    training_records.append(score)
    running_score = running_score * 0.99 + score * 0.01
    running_score_records.append(running_score)

    # 每 15 个 episode 显示进度并评估
    if (i_ep + 1) % 15 == 0:
        print('Ep {}\tLast score: {:.2f}\tMoving average score: {:.2f}'.format(i_ep, score, running_score))

        c1_score = eval(agent, target_env[0])
        c2_score = eval(agent, target_env[1])
        c1_training_records.append(c1_score)
        c2_training_records.append(c2_score)

        print('c1 score: {:.2f}\t c2 score: {:.2f}'.format(c1_score, c2_score))

        # 记录训练数据
        logger.log_training_step(
            episode=i_ep,
            score=score,
            running_score=running_score,
            c1_score=c1_score,
            c2_score=c2_score
        )
        
        # 记录测试结果（JSON格式）
        logger.log_test_result(
            test_name=f"episode_{i_ep}",
            results={
                "source_env_score": float(score),
                "target_env_c1_score": float(c1_score),
                "target_env_c2_score": float(c2_score),
                "mean_target_score": float((c1_score + c2_score) / 2),
                "running_score": float(running_score)
            },
            metadata={"episode": i_ep}
        )
        
        # 保存测试数据（每15个episode保存一次）
        logger.save_test_data(f"test_ep{i_ep:04d}.json")

        # 绘制训练曲线
        f, axs = plt.subplots(2, 2, figsize=(16, 8))
        axs[0][0].plot(range(len(training_records)), training_records)
        axs[0][0].set_title("Baseline Training Records")
        axs[0][1].plot(range(len(running_score_records)), running_score_records)
        axs[0][1].set_title("Baseline Running Score")
        axs[1][0].plot(range(len(c1_training_records)), c1_training_records)
        axs[1][0].set_title("Baseline C1 Training Records")
        axs[1][1].plot(range(len(c2_training_records)), c2_training_records)
        axs[1][1].set_title("Baseline C2 Training Records")

        f.savefig('./output_r/baseline_result_%04d.png' % i_ep)
        plt.close(f)
        
        # 保存检查点和训练数据（每100个episode保存一次）
        if (i_ep + 1) % 100 == 0:
            logger.save_checkpoint(net, optimizer, i_ep)
            logger.save_training_data(f"training_ep{i_ep:04d}.json")
            print(f"💾 已保存检查点和训练数据到 episode {i_ep}")

# 训练结束，保存最终数据
logger.save_training_data("training_final.json")
logger.save_test_data("test_final.json")

# 保存最终检查点
logger.save_checkpoint(net, optimizer, 2999, "checkpoint_final.pth")

# 打印实验摘要
summary = logger.get_experiment_summary()
print("\n" + "="*50)
print("实验摘要:")
print(json.dumps(summary, indent=2, ensure_ascii=False))
print("="*50)

## 7. 训练完成

训练完成后，可以在 `./output_r/` 目录中查看生成的图像和训练曲线。

**对比说明**：可以将 baseline 的结果与 DANN 方法的结果进行对比，评估域适应的有效性。